In [30]:
import os
import tensorflow as tf
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.layers import Dense, GlobalAveragePooling2D, Conv2D, Input
from keras import callbacks, optimizers
import numpy as np

In [6]:
# from archive.translate import translate

In [ ]:
# for i in os.listdir("archive/raw-img"):
    # os.rename("archive/raw-img/" + i, "archive/raw-img/" + translate[i])
    # print(i)

# for i in os.listdir("archive/raw-img"):
#     print(i, len(os.listdir(f"archive/raw-img/{i}")))

In [34]:
try:
    os.mkdir("train")
    os.mkdir("test")
except:
    pass
for i in os.listdir(f"archive/raw-img"):
    try:
        os.mkdir(f"train/{i}")
        os.mkdir(f"test/{i}")
    except:
        pass
    for j in os.listdir(f"archive/raw-img/{i}")[:1000]:
        os.rename(f"archive/raw-img/{i}/{j}", f"train/{i}/{j}")
    for j in os.listdir(f"archive/raw-img/{i}")[:400]:
        os.rename(f"archive/raw-img/{i}/{j}", f"test/{i}/{j}")

In [35]:
def image_data(dir_path, target_size, batch, class_list, preprocessing):
    if preprocessing:
        gen_object = ImageDataGenerator(preprocessing_function=preprocessing)
    else:
        gen_object = ImageDataGenerator()

    return gen_object.flow_from_directory(dir_path, target_size=target_size, batch_size=batch, class_mode="sparse", classes=class_list, shuffle=True)


In [36]:
train_data_gen = image_data("train", (224, 224), 500, os.listdir("train"), preprocess_input)
valid_data_gen = image_data("test", (224, 224), 500, os.listdir("test"), preprocess_input)

Found 19542 images belonging to 10 classes.
Found 6637 images belonging to 10 classes.


In [37]:
base_model = MobileNetV2(input_shape=(224, 224, 3), alpha=1.0, include_top=False, weights="imagenet", input_tensor=None, pooling=None, classes=1000, classifier_activation="softmax")

In [38]:
base_model.trainable = False

In [39]:
from keras.models import Sequential
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation="relu"))
model.add(Dense(10, activation="softmax"))

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [40]:
early_stopping = callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min")
save_checkpoint = callbacks.ModelCheckpoint("mdl_wts.hdf5", save_best_only=True, monitor="val_loss", mode="min")

In [21]:
model.fit(train_data_gen, batch_size=500, validation_data=valid_data_gen, callbacks=[early_stopping, save_checkpoint], epochs=10)

Epoch 1/10
20/20 [==============================] - 719s 33s/step - loss: 0.4414 - accuracy: 0.8667 - val_loss: 0.2417 - val_accuracy: 0.9332
Epoch 2/10
20/20 [==============================] - 355s 18s/step - loss: 0.1159 - accuracy: 0.9654 - val_loss: 0.1925 - val_accuracy: 0.9442
Epoch 3/10
20/20 [==============================] - 414s 21s/step - loss: 0.0725 - accuracy: 0.9786 - val_loss: 0.1900 - val_accuracy: 0.9417
Epoch 4/10
20/20 [==============================] - 344s 17s/step - loss: 0.0489 - accuracy: 0.9886 - val_loss: 0.2015 - val_accuracy: 0.9439
Epoch 5/10
20/20 [==============================] - 347s 17s/step - loss: 0.0348 - accuracy: 0.9933 - val_loss: 0.1978 - val_accuracy: 0.9433
Epoch 6/10
20/20 [==============================] - 341s 17s/step - loss: 0.0258 - accuracy: 0.9953 - val_loss: 0.2073 - val_accuracy: 0.9446
Epoch 7/10
20/20 [==============================] - 341s 17s/step - loss: 0.0177 - accuracy: 0.9976 - val_loss: 0.2044 - val_accuracy: 0.9464
Epoch 